In [ ]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torchvision.transforms.functional as F
import random
try:
    import timm 
except ImportError:
    print("Устанавливаем библиотеку timm...")
    import subprocess
    subprocess.check_call(["pip", "install", "timm"])
    import timm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm
import copy
import time
import timm

In [ ]:
# CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
DATA_DIR = '/kaggle/input/colors/dataset_colors'
TRAIN_DATA_DIR = '/kaggle/input/colors/dataset_colors/train_data'
TEST_DATA_DIR = '/kaggle/input/colors/dataset_colors/test_data'
TRAIN_CSV = '/kaggle/input/colors/dataset_colors/train_data.csv'
TEST_CSV = '/kaggle/input/colors/dataset_colors/test_data.csv'

In [ ]:
TRANSLIT_TO_RU = {
    'bezhevyi': 'бежевый',
    'belyi': 'белый',
    'biryuzovyi': 'бирюзовый',
    'bordovyi': 'бордовый',
    'goluboi': 'голубой',
    'zheltyi': 'желтый',
    'zelenyi': 'зеленый',
    'zolotoi': 'золотой',
    'korichnevyi': 'коричневый',
    'krasnyi': 'красный',
    'oranzhevyi': 'оранжевый',
    'raznocvetnyi': 'разноцветный',
    'rozovyi': 'розовый',
    'serebristyi': 'серебряный',
    'seryi': 'серый',
    'sinii': 'синий',
    'fioletovyi': 'фиолетовый',
    'chernyi': 'черный'
}

In [ ]:
# Маппинг цветов
COLORS = {
    'бежевый': 'beige',
    'белый': 'white',
    'бирюзовый': 'turquoise',
    'бордовый': 'burgundy',
    'голубой': 'blue',
    'желтый': 'yellow',
    'зеленый': 'green',
    'золотой': 'gold',
    'коричневый': 'brown',
    'красный': 'red',
    'оранжевый': 'orange',
    'разноцветный': 'variegated',
    'розовый': 'pink',
    'серебряный': 'silver',
    'серый': 'gray',
    'синий': 'blue',
    'фиолетовый': 'purple',
    'черный': 'black'
}

# Создаем обратный словарь для получения индекса по цвету
COLOR_INDICES = {c: i for i, c in enumerate(COLORS.keys())}

CATEGORIES = ['одежда для девочек', 'столы', 'стулья', 'сумки']

In [ ]:
class ProductDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_test=False):
        self.df = df.reset_index(drop=True)  # Сбрасываем индексы после фильтрации
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.category_to_idx = {cat: idx for idx, cat in enumerate(CATEGORIES)}
        self.color_to_idx = {color: idx for idx, color in enumerate(COLORS.keys())}
        
        # Проверяем все пути к изображениям заранее
        self.valid_indices = []
        for idx in range(len(df)):
            img_path = os.path.join(self.img_dir, f"{df.iloc[idx]['id']}.jpg")
            if os.path.exists(img_path):
                self.valid_indices.append(idx)
        
        if len(self.valid_indices) == 0:
            raise ValueError(f"Не найдено ни одного изображения в директории {img_dir}")
        
        print(f"Найдено {len(self.valid_indices)} валидных изображений из {len(df)}")
        
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_id = self.df.iloc[real_idx]['id']
        img_path = os.path.join(self.img_dir, f"{img_id}.jpg")
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Ошибка при загрузке изображения {img_path}: {str(e)}")
            raise
        
        if self.transform:
            try:
                image = self.transform(image)
            except Exception as e:
                print(f"Ошибка при применении трансформации к {img_path}: {str(e)}")
                raise
            
        category = self.category_to_idx[self.df.iloc[real_idx]['category']]
        
        if not self.is_test:
            color_translit = self.df.iloc[real_idx]['target']
            color_ru = TRANSLIT_TO_RU[color_translit]
            color = self.color_to_idx[color_ru]
            return image, category, color
        
        return image, category, img_id

In [ ]:
class ColorClassifier(nn.Module):
    def __init__(self, num_colors, num_categories):
        super().__init__()
        # Используем более легкий и быстрый вариант ViT
        self.backbone = timm.create_model(
            'beitv2_large_patch16_224', 
            pretrained=True, 
            num_classes=0,  # Без верхнего слоя классификации
        )
        
        # Фиксируем большую часть весов для ускорения обучения
        for param in list(self.backbone.parameters())[:-30]:
            param.requires_grad = False
            
        # Расширение для быстрой инференции с помощью кэширования
        self.backbone.reset_classifier(0)
        
        # Размерность характеристик модели
        self.feature_dim = self.backbone.embed_dim  # Для vit_tiny это 192
        
        # Эмбеддинг категории
        self.category_embedding = nn.Embedding(num_categories, 32)
        
        # Классификационная голова
        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim + 32, 256),  # Меньше параметров для ускорения
            nn.ReLU(),
            nn.Dropout(0.2),  # Меньше дропаут для более быстрой сходимости
            nn.Linear(256, num_colors)
        )
        
        # Для оптимизации torch.jit
        self.example_input = torch.zeros(1, 3, 224, 224)
        self.example_category = torch.LongTensor([0])
        
    def forward(self, x, category):
        features = self.backbone(x)
        
        category_emb = self.category_embedding(category)
        combined = torch.cat([features, category_emb], dim=1)
        
        return self.classifier(combined)

    def optimize_for_inference(self, device):
        """Оптимизирует модель для быстрой инференции"""
        self.eval()
        # Использование TorchScript для оптимизации
        return torch.jit.trace((self.example_input.to(device), self.example_category.to(device)), self)

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import copy

# Импортируем метрики из scikit-learn
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    num_epochs=400,
    seed=42
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    scaler = torch.cuda.amp.GradScaler()  # Для автоматического масштабирования градиентов при mixed precision
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': []
    }
    
    for epoch in range(num_epochs):
        # Устанавливаем сид для каждой эпохи для воспроизводимости
        # Используем разные сиды для разных эпох, но детерминированно
        epoch_seed = seed + epoch
        torch.manual_seed(epoch_seed)
        np.random.seed(epoch_seed)
        random.seed(epoch_seed)
        
        print(f'Эпоха {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Обучение
        model.train()
        running_loss = 0.0
        
        train_batches = tqdm(train_loader, desc="Обучение")
        
        for inputs, categories, colors in train_batches:
            inputs = inputs.to(device)
            categories = categories.to(device)
            colors = colors.to(device)
            
            # Обнуляем градиенты
            optimizer.zero_grad()
            
            # Прямой проход с автоматическим приведением к float16
            with torch.cuda.amp.autocast():
                outputs = model(inputs, categories)
                loss = criterion(outputs, colors)
            
            # Обратное распространение с масштабированием для предотвращения исчезновения градиентов
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item() * inputs.size(0)
            
            # Обновляем статус прогресс-бара
            train_batches.set_postfix(loss=loss.item())
        
        epoch_loss = running_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_loss)
        
        print(f'Потери при обучении: {epoch_loss:.4f}')
        
        # Валидация
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, categories, colors in tqdm(val_loader, desc="Валидация"):
                inputs = inputs.to(device)
                categories = categories.to(device)
                colors = colors.to(device)
                
                # Используем mixed precision и для инференса
                with torch.cuda.amp.autocast():
                    outputs = model(inputs, categories)
                    loss = criterion(outputs, colors)
                
                val_loss += loss.item() * inputs.size(0)
                
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(colors.cpu().numpy())
        
        val_loss = val_loss / len(val_loader.dataset)
        
        # Вычисляем метрики
        val_acc = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(f1)
        
        print(f'Потери при валидации: {val_loss:.4f}')
        print(f'Точность: {val_acc:.4f}')
        print(f'F1-мера: {f1:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        
        # Save the best model based on F1 score
        if f1 > best_f1:  # Change best_acc to best_f1 for clarity
            best_f1 = f1  # Update best_acc to best_f1
            best_model_wts = copy.deepcopy(model.state_dict())
            # Save the intermediate best model
            torch.save({
                'model_state_dict': model.state_dict(),
                'num_colors': len(COLORS),
                'num_categories': len(CATEGORIES)
            }, 'vit_color_classifier.pth')
            print("Сохранена новая лучшая модель по F1!")
        
        # Шаг планировщика скорости обучения
        scheduler.step()
        
        print()
    
    print(f'Лучшая точность при валидации: {best_f1:.4f}')
    
    # Возвращаем лучшие веса модели и историю обучения
    return best_model_wts, history

In [ ]:
def load_model(model_path):
    """
    Загружает ранее обученную модель из указанного пути.
    Если модель была сохранена как TorchScript, загрузит её как таковую.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Проверяем формат сохраненной модели
    try:
        # Пробуем загрузить как TorchScript модель
        model = torch.jit.load(model_path, map_location=device)
        print("Загружена оптимизированная TorchScript модель")
        return model
    except:
        # Загружаем как обычную модель
        print("Загружаем модель из стандартных весов...")
        model = ColorClassifier(len(COLORS), len(CATEGORIES))
        
        checkpoint = torch.load(model_path, map_location=device)
        if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
        else:
            model.load_state_dict(checkpoint)
        
        # Установка модели в режим оценки (инференса)
        model.eval()
        
        # Если мы на GPU, оптимизируем модель для инференса
        if device.type == 'cuda' and hasattr(model, 'optimize_for_inference'):
            try:
                print("Оптимизация модели для быстрого инференса...")
                model = model.optimize_for_inference(device)
                # Сохраняем оптимизированную версию
                optimized_path = model_path.replace('.pth', '_optimized.pth')
                torch.jit.save(model, optimized_path)
                print(f"Оптимизированная модель сохранена как {optimized_path}")
            except Exception as e:
                print(f"Не удалось оптимизировать модель: {e}")
        
        return model

In [ ]:
def predict(model, test_loader, seed=42):
    # Устанавливаем сид для воспроизводимости
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    model = model.to(device)
    model.eval()
    
    # Оптимизация модели для инференции
    if hasattr(model, 'optimize_for_inference') and device.type == 'cuda':
        try:
            model = model.optimize_for_inference(device)
        except Exception as e:
            print(f"Не удалось оптимизировать модель: {e}")
    
    predictions = []
    ids = []
    
    color_list = list(COLORS.keys())
    
    # Замер времени на один батч
    batch_times = []
    
    with torch.no_grad():
        for images, categories, img_ids in test_loader:
            images = images.to(device)
            categories = categories.to(device)
            
            # Замер времени
            start_time = time.time()
            outputs = model(images, categories)
            end_time = time.time()
            
            inference_time = (end_time - start_time) * 1000  # мс
            batch_times.append(inference_time / len(images))  # время на один образец
            
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            
            for img_id, prob in zip(img_ids, probs):
                pred_dict = {color: float(p) for color, p in zip(color_list, prob)}
                pred_color = color_list[np.argmax(prob)]
                
                predictions.append({
                    'id': img_id,
                    'predict_proba': json.dumps(pred_dict),
                    'predict_color': pred_color
                })
                
    # Вывод среднего времени инференции
    if batch_times:
        avg_time = sum(batch_times) / len(batch_times)
        print(f"Среднее время инференции на одно изображение: {avg_time:.2f} мс")
    
    # Создаем DataFrame с предсказаниями
    predictions_df = pd.DataFrame(predictions)
    return predictions_df

In [ ]:
def benchmark_inference(model, image_size=(224, 224), num_runs=100, batch_size=1):
    """
    Тестирует производительность модели, измеряя время инференса.
    
    Args:
        model: Модель для тестирования
        image_size: Размер входного изображения (высота, ширина)
        num_runs: Количество прогонов для усреднения результатов
        batch_size: Размер батча для тестирования
        
    Returns:
        float: Среднее время инференса в миллисекундах на один образец
    """
    device = next(model.parameters()).device
    
    # Устанавливаем сид для воспроизводимости генерации тестовых данных
    torch.manual_seed(42)
    
    # Создаем случайные тензоры для тестирования
    dummy_input = torch.randn(batch_size, 3, *image_size, device=device)
    dummy_category = torch.zeros(batch_size, dtype=torch.long, device=device)
    
    # Прогрев модели
    print("Прогрев модели...")
    with torch.no_grad():
        for _ in range(10):
            _ = model(dummy_input, dummy_category)
    
    # Синхронизация GPU перед тестами
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # Измерение времени
    print(f"Запуск бенчмарка ({num_runs} прогонов)...")
    times = []
    
    with torch.no_grad():
        for _ in range(num_runs):
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.time()
            _ = model(dummy_input, dummy_category)
            
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            end_time = time.time()
            inference_time = (end_time - start_time) * 1000  # мс
            times.append(inference_time / batch_size)  # время на один образец
    
    # Вычисляем статистики
    avg_time = sum(times) / len(times)
    min_time = min(times)
    max_time = max(times)
    p95_time = sorted(times)[int(len(times) * 0.95)]
    
    print(f"Результаты бенчмарка:")
    print(f"  Среднее время инференса: {avg_time:.2f} мс")
    print(f"  Минимальное время: {min_time:.2f} мс")
    print(f"  Максимальное время: {max_time:.2f} мс")
    print(f"  95-й перцентиль: {p95_time:.2f} мс")
    
    return avg_time

def set_seed(seed=42):
    """
    Set all random seeds for reproducibility.
    
    Args:
        seed (int): Seed value to use for all random number generators
        
    This function sets seeds for:
    - Python's random module
    - NumPy
    - PyTorch (both CPU and CUDA)
    - CUDA operations
    
    It also configures PyTorch to use deterministic algorithms where possible.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    
    # Make operations deterministic
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Set environment variable for Python hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    print(f"All random seeds set to {seed} for reproducibility")

def main():
    # Set random seeds for reproducibility
    set_seed(42)
    
    # Устанавливаем все необходимые настройки для максимальной производительности
    # Note: Setting cudnn.benchmark=True can improve performance but reduces reproducibility
    # We'll comment this out since we prioritize reproducibility
    # torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    # Проверяем наличие библиотеки timm
    try:
        import timm
        print(f"Версия timm: {timm.__version__}")
    except ImportError:
        print("Устанавливаем библиотеку timm...")
        import subprocess
        subprocess.check_call(["pip", "install", "timm"])
        import timm
        print(f"Установлена библиотека timm версии {timm.__version__}")
    
    required_paths = [
        TRAIN_CSV,
        TEST_CSV,
        TRAIN_DATA_DIR,
        TEST_DATA_DIR
    ]
    
    for path in required_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Не найден путь: {path}")
            
    print("Все необходимые файлы и директории найдены")
    
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    
    print(f"Исходный размер тренировочного датасета: {len(train_df)}")
    print(f"Исходный размер тестового датасета: {len(test_df)}")
    
    def check_image_exists(row, data_dir):
        img_path = os.path.join(data_dir, f"{row['id']}.jpg")
        return os.path.exists(img_path)
    
    # Фильтруем только существующие изображения
    train_df = train_df[train_df.apply(lambda row: check_image_exists(row, TRAIN_DATA_DIR), axis=1)]
    test_df = test_df[test_df.apply(lambda row: check_image_exists(row, TEST_DATA_DIR), axis=1)]
    
    print(f"Размер тренировочного датасета после фильтрации: {len(train_df)}")
    print(f"Размер тестового датасета после фильтрации: {len(test_df)}")
    
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
    
    print(f"Размер обучающего датасета: {len(train_df)}")
    print(f"Размер валидационного датасета: {len(val_df)}")
    
    # Оптимизированные трансформации для ViT
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    # Обучающие трансформации с фиксированными сидами для воспроизводимости
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Сразу переходим к целевому размеру для ускорения 
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10, fill=0, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    # Облегченные трансформации для тестирования и валидации
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Целевой размер для ViT
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    
    # Создаем датасеты
    train_dataset = ProductDataset(train_df, TRAIN_DATA_DIR, transform=train_transform)
    val_dataset = ProductDataset(val_df, TRAIN_DATA_DIR, transform=test_transform)
    test_dataset = ProductDataset(test_df, TEST_DATA_DIR, transform=test_transform, is_test=True)
    
    # Функция инициализации воркеров для обеспечения воспроизводимости
    def seed_worker(worker_id):
        worker_seed = 42 + worker_id
        np.random.seed(worker_seed)
        random.seed(worker_seed)
        
    # Генератор для DataLoader
    g = torch.Generator()
    g.manual_seed(42)
    
    # Оптимизируем загрузчики данных
    # Используем pin_memory=True для ускорения передачи данных на GPU
    # Увеличиваем num_workers для параллельной загрузки данных
    num_workers = min(4, os.cpu_count() or 4)  # Разумное количество потоков
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=32, 
        shuffle=True, 
        num_workers=num_workers,
        pin_memory=True,
        worker_init_fn=seed_worker,
        generator=g
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=64,  # Больший размер батча для валидации
        shuffle=False, 
        num_workers=num_workers,
        pin_memory=True,
        worker_init_fn=seed_worker,
        generator=g
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=64,  # Используем больший размер батча для ускорения инференса
        shuffle=False, 
        num_workers=num_workers,
        pin_memory=True,
        worker_init_fn=seed_worker,
        generator=g
    )
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    torch.cuda.empty_cache()  # Очищаем память GPU
    
    model_path = "/kaggle/input/macro_/pytorch/default/1/macro_weights.pth"
    
    if os.path.exists(model_path):
        print("Загружаем существующую модель...")
        model = load_model(model_path)
        model = model.to(device)
    else:
        print("Начинаем обучение новой модели...")
        # Устанавливаем сид перед инициализацией модели для воспроизводимости весов
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        # Если используется timm, установим его сид тоже
        try:
            import timm
            timm.random.set_random_seed(42)
        except (ImportError, AttributeError):
            pass
            
        model = ColorClassifier(len(COLORS), len(CATEGORIES))
        model = model.to(device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        
        # --- ВАЖНО: теперь train_model возвращает два значения ---
        best_model_state, history = train_model(
            model, train_loader, val_loader, criterion, optimizer, scheduler, seed=42
        )
        model.load_state_dict(best_model_state)
    
    # Выполняем прогрев модели для оптимизации CUDA графов
    if device.type == 'cuda':
        print("Оптимизация модели для инференса...")
        # Устанавливаем сид для воспроизводимости генерации тестовых данных
        torch.manual_seed(42)
        dummy_input = torch.randn(1, 3, 224, 224, device=device)
        dummy_category = torch.tensor([0], device=device)
        with torch.no_grad():
            for _ in range(10):  # Прогрев
                _ = model(dummy_input, dummy_category)
    
    # Запускаем бенчмарк для измерения производительности
    print("\nЗапуск бенчмарка производительности...")
    avg_time = benchmark_inference(model, image_size=(224, 224), num_runs=100)
    
    if avg_time <= 200:
        print(f"✅ Модель соответствует требованию по скорости (< 200 мс): {avg_time:.2f} мс")
    else:
        print(f"❌ Модель НЕ соответствует требованию по скорости (< 200 мс): {avg_time:.2f} мс")
    
    print("\nДелаем предсказания...")
    predictions_df = predict(model, test_loader, seed=42)
    predictions_df.to_csv('submission.csv', index=False)
    print("Готово! Результаты сохранены в submission.csv")

In [ ]:
if __name__ == '__main__':
    main()